In [5]:
!python preprocess_data.py --help

Usage: preprocess_data.py [OPTIONS]

Options:
  --raw_data_path TEXT  Location where the raw NYC taxi trip data was saved
  --dest_path TEXT      Location where the resulting files will be saved
  --help                Show this message and exit.


In [7]:
!mkdir processed_data/

In [8]:
!python preprocess_data.py --raw_data_path ../data/ --dest_path processed_data/

In [9]:
!python train.py --help

Usage: train.py [OPTIONS]

Options:
  --data_path TEXT  Location where the processed NYC taxi trip data was saved
  --help            Show this message and exit.


In [10]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [13]:
!python train.py --data_path processed_data/

2024/06/01 21:05:14 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/06/01 21:05:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c3b73f8271d143bbb36a9e55c5ad4100', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/06/01 21:05:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/couzhei/.pyenv/versions/3.10.14/envs/mle/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [14]:
!python hpo.py --help

2024/06/01 21:56:29 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
Usage: hpo.py [OPTIONS]

Options:
  --data_path TEXT      Location where the processed NYC taxi trip data was
                        saved
  --num_trials INTEGER  The number of parameter evaluations for the optimizer
                        to explore
  --help                Show this message and exit.


In [16]:
!python hpo.py --data_path processed_data/ --num_trials 80

100%|██████████| 80/80 [10:40<00:00,  8.01s/trial, best loss: 5.314362499822815]


In [17]:
!python register_model.py --help

2024/06/01 22:18:45 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
Usage: register_model.py [OPTIONS]

Options:
  --data_path TEXT  Location where the processed NYC taxi trip data was saved
  --top_n INTEGER   Number of top models that need to be evaluated to decide
                    which one to promote
  --help            Show this message and exit.


In [21]:

!python register_model.py --data_path processed_data/ --top_n 5

In [30]:
import pickle, os, mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.sklearn.autolog()


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

logged_model = 'runs:/f6408f5728d74b26b45b36206f8f1217/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
from sklearn.metrics import mean_squared_error

X_test, y_test = load_pickle(os.path.join("processed_data/", "test.pkl"))
mean_squared_error(y_test, loaded_model.predict(X_test))

30.780442361479178